In [3]:
import xlsxwriter as xls 
import numpy as np
import pandas as pd
import pyodbc
import os
import re

In [3]:
keys=pd.read_excel('SDGs關鍵字總表.xlsx', engine='openpyxl')
writer = pd.ExcelWriter('SDGs關鍵字整理.xlsx')

for col in keys.columns:
    temp=keys[col]
    temp=temp.drop_duplicates()
    temp.to_excel(writer,sheet_name = col,index=False)
writer.save()

In [14]:
# 因為已經有 'SDGs 課程分類_統整.xlsx' 這個檔案了，所以可以不用執行。
# 此程式碼中的 'SDGs課程分類_婷茹.xlsx' 等檔案來已有變更，以 'SDGs 課程分類_統整.xlsx' 中的資料為主。

col=['PK','SDG1：終結貧窮.1',          'SDG2：終結飢餓.1',
             'SDG3：良好健康與福祉.1',          'SDG4：優質教育.1',          'SDG5：性別平等.1',
            'SDG6：潔淨水資源與衛生.1',    'SDG7：人人可負擔的永續能源.1',     'SDG8：良好工作及經濟成長.1',
         'SDG9：工業化、創新及基礎建設.1',        'SDG10：消弭不平等.1',      'SDG11：永續城鄉與社區.1',
         'SDG12：負責任的生產消費循環.1',       'SDG13：氣候變遷對策.1',         'SDG14：海洋生態.1',
               'SDG15：陸域生態.1',     'SDG16：公平、正義與和平.1',       'SDG17：全球夥伴關係.1']
t1=pd.read_excel('SDGs課程分類_婷茹.xlsx', engine='openpyxl')
t2=pd.read_excel('SDGs課程分類_昱霖.xlsx', engine='openpyxl')
t3=pd.read_excel('SDGs課程分類_聖晴.xlsx',sheet_name='總表', engine='openpyxl')

t4=pd.merge(t3,t2[col],how='left',on='PK')
t5=pd.merge(t4,t1[col],how='left',on='PK')
print(len(t5))


In [15]:
# 同上

t5.to_excel('SDGs課程分類_統整.xlsx')

NameError: name 't5' is not defined

In [4]:
# 辨別課程名稱、大綱等是否包含中文。

def is_contains_chinese(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False

In [37]:
# 運用關鍵字分類課程，在與人工分類的結果進行比對。
# TODO: 外語課程的課程綱要可能使用中文、英文外的語言撰寫課程綱要，我們拿到的資料中已經是亂碼了。

keyword = pd.read_excel('SDGs關鍵字總表.xlsx',sheet_name='關鍵字(移除重複)')
classified_by_human = pd.read_excel('SDGs課程分類_統整.xlsx')
course = pd.read_excel('SDGs課程分類_統整(測試).xlsx')
keyflag = True

# 建一張 map 紀錄每個 keywaord 被使用的次數，與使用時判斷正確與否。
keyword_log = {}

#print(course)
#print(keyword.columns)

for SDG in keyword.columns:
    print("<<<<<<<<<<<<<   ", SDG, "  >>>>>>>>>>>>>>>>")
    temp = keyword[SDG].dropna()
    for key in temp:
        keyflag = True
        key = str.lower(key)
        
        # 在 keyword_log 中用 list 的格式新增這個 key 的紀錄，其中每一個 index 就是這個 key 判斷這 index 的課程時的結果。
        current_key_log = []
        for course_num, name, dep, schedule, syllabus, style, schema in zip(course['PK'],course['科目名稱'].fillna('無'), course['主開學系'].fillna('無'), course['每周進度'].fillna('無'), course['概述與目標'].fillna('無'), course['教學方法'].fillna('無'), course['課程規劃'].fillna('無')):
            name = str.lower(name)
            schedule = str.lower(schedule)
            syllabus = str.lower(syllabus)
            style = str.lower(style)
            schema = str.lower(schema)
            if '&' in key:
                flag = True
                sub_keys = re.split("&", key)
                for sub_key in sub_keys:
                    sub_key = sub_key.strip()
                    if (is_contains_chinese(key) == False):
                        sub_key = sub_key + ' '
                    
                        #if keyflag:
                            #print(sub_key)
                    flag = flag and ((sub_key in name) or (sub_key in schedule) or (sub_key in syllabus) or (sub_key in style) or (sub_key in schema))
                
                keyflag = False
                
                if flag:
                    # 成功比對，課程內容中含有此關鍵字，將其加入資料中。
                    #course.loc[course['PK'] == course_num, SDG + '_程式'] = key #將其記錄到 excel 檔案中。
                    
                    # 檢查人工判斷時的結果是否是此 SDGs 標籤。非零代表有人也認為這門課符合這個 SDGs 標籤。
                    # append 的值中：0 表示此關鍵字沒有被使用，1 表示被使用蛋判斷錯誤，2 表示有使用且判斷正確。
                    if int(classified_by_human.loc[classified_by_human['PK'] == course_num, SDG + '(統整)']) > 0:
                        current_key_log.append(2)
                    else :
                        current_key_log.append(1)
                else :
                    current_key_log.append(0)
                    
            else:
                key=key.strip()
                if is_contains_chinese(key) == False:
                    key = key + ' '
                    
                if keyflag:
                    print(key)
                    keyflag = False
                    
                if (key in name) or (key in schedule) or (key in syllabus) or (key in style) or (key in schema):
                    # 成功比對，課程內容中含有此關鍵字，將其加入資料中。
                    #course.loc[course['PK'] == course_num,SDG+'_程式'] = key
                    
                    # 檢查人工判斷時的結果是否是此 SDGs 標籤。非零代表有人也認為這門課符合這個 SDGs 標籤。
                    # append 的值中：0 表示此關鍵字沒有被使用，1 表示被使用蛋判斷錯誤，2 表示有使用且判斷正確。
                    if int(classified_by_human.loc[classified_by_human['PK'] == course_num, SDG + '(統整)']) > 0:
                        current_key_log.append(2)
                    else :
                        current_key_log.append(1)
                else :
                    current_key_log.append(0)
            #ok
            if SDG == 'SDG4：優質教育' and dep == '師資培育中心':
                    course.loc[course['PK'] == course_num,SDG+'_程式'] = '師培中心開設課程'
            if SDG == 'SDG4：優質教育' and dep == '語言教學與研究中心':
                    course.loc[course['PK'] == course_num,SDG+'_程式'] = '語言教學與研究中心課程'
            if SDG == 'SDG4：優質教育' and dep == '寫作中心':
                    course.loc[course['PK'] == course_num,SDG + '_程式'] = '寫作中心'
        keyword_log[key] = current_key_log
        print(key, "test")
print("%%%%")
#d=pd.read_excel('SDG/'+school+'課程分類結果.xlsx')

#temp=pd.merge(course,d,how='left',on='PK')

#temp=temp.rename(columns={'學年度_x':'學年度','學期別_x':'學期別','課號_x':'課號','科目代碼_x':'科目代碼','科目名稱_x':'科目名稱','課程目標_x':'課程目標','課程進度_x':'課程進度','課程大綱_x':'課程大綱','開課學院_x':'開課學院','主開學系_x':'主開學系'})
#temp[['學年度','學期別','課號','科目代碼','科目名稱','課程目標','課程進度','課程大綱','開課學院','主開學系','SDG1：終結貧窮_x','SDG1：終結貧窮_y','SDG2：終結飢餓_x','SDG2：終結飢餓_y','SDG3：良好健康與福祉_x','SDG3：良好健康與福祉_y','SDG4：優質教育_x','SDG4：優質教育_y','SDG5：性別平等_x','SDG5：性別平等_y','SDG6：潔淨水資源與衛生_x','SDG6：潔淨水資源與衛生_y','SDG7：人人可負擔的永續能源_x','SDG7：人人可負擔的永續能源_y','SDG8：良好工作及經濟成長_x','SDG8：良好工作及經濟成長_y','SDG9：工業化、創新及基礎建設_x','SDG9：工業化、創新及基礎建設_y','SDG10：消弭不平等_x','SDG10：消弭不平等_y','SDG11：永續城鄉與社區_x','SDG11：永續城鄉與社區_y','SDG12：負責任的生產消費循環_x','SDG12：負責任的生產消費循環_y','SDG13：氣候變遷對策_x','SDG13：氣候變遷對策_y','SDG14：海洋生態_x','SDG14：海洋生態_y','SDG15：陸域生態_x','SDG15：陸域生態_y','SDG16：公平、正義與和平_x','SDG16：公平、正義與和平_y','SDG17：全球夥伴關係_x','SDG17：全球夥伴關係_y']].to_excel('SDG/'+school+'程式分類_名稱大綱進度.xlsx',index=False)
#course.to_excel(SDGs課程分類_統整(含程式.xlsx',index=False)
course.to_excel('SDGs課程分類_統整(測試).xlsx', index = False)
print('done')

<<<<<<<<<<<<<    SDG1：終結貧窮   >>>>>>>>>>>>>>>>
imf 
imf  test
microeconomics 
microeconomics  test
poverty 
poverty  test
women’s poverty 
women’s poverty  test
公益服務
公益服務 test
公益活動
公益活動 test
分析社會現象
分析社會現象 test
失業&物價膨脹 test
市場消費
市場消費 test
市場經濟
市場經濟 test
全球成長不均
全球成長不均 test
回饋社會
回饋社會 test
在地關懷
在地關懷 test
老人&生活品質 test
志工服務
志工服務 test
志工計畫
志工計畫 test
服務&兒童&關懷&陪讀 test
服務&兒童&關懷&課輔 test
服務&原住民&助教 test
服務&原住民&課輔 test
服務&弱勢&關懷 test
社會&關懷 test
社會公平正義
社會公平正義 test
社會階級&貧富差距 test
社會資源分配
社會資源分配 test
社會福利政策
社會福利政策 test
社會關懷
社會關懷 test
長期照護
長期照護 test
長照
長照 test
長照&社會福利 test
長照2.0
長照2.0 test
食物銀行
食物銀行 test
個體經濟
個體經濟 test
剝削勞工
剝削勞工 test
弱勢&同理心&關懷 test
弱勢者
弱勢者 test
弱勢族群
弱勢族群 test
弱勢關懷
弱勢關懷 test
財政政策
財政政策 test
財富不均
財富不均 test
逃脫貧窮
逃脫貧窮 test
偏鄉&健康照護 test
健康保險
健康保險 test
貧困者扶助
貧困者扶助 test
貧富不均
貧富不均 test
貧富差距
貧富差距 test
貧窮&社會救助 test
貧窮&糧食危機 test
貧窮&難民 test
貧窮化&經濟轉型&資本主義 test
貨幣&財政 test
勞動保護
勞動保護 test
勞動權益
勞動權益 test
發展中國家的債務與貧富差距
發展中國家的債務與貧富差距 test
鄉村銀行
鄉村銀行 test
愛心&課輔 test
慈善團體
慈善團體 test
經濟學&財政 test
經濟學&貨幣 test
經濟學&總體

健康&照護&設計 test
健康&照護&創新 test
健康&照護&應用 test
健康&衛生教育 test
健康&臨床&個案 test
健康&臨床實務 test
健康&醫療&實務 test
健康&議題&傳播 test
健康&護理&實習 test
健康生活型態
健康生活型態 test
健康制度
健康制度 test
健康促進&政策 test
健康促進&應用 test
健康風險&藥物 test
健康風險評估
健康風險評估 test
健康食品&衛生安全 test
健康傳播
健康傳播 test
健康傳播&健康不平等 test
健康照護
健康照護 test
健康照護&心理疾患 test
健康照護&個案設計 test
健康照護&設計 test
健康照護&實務&政策 test
健康照護&實習 test
健康照護政策&藥物管理 test
健康照顧
健康照顧 test
健康福祉
健康福祉 test
健康議題
健康議題 test
培育&護理人才 test
培植&醫師 test
培養&健康照護人員 test
排放水處理
排放水處理 test
探討&情緒&健康 test
探討&應用&疾病治療 test
粒狀污染物
粒狀污染物 test
細胞治療
細胞治療 test
規律運動
規律運動 test
設計&公共衛生 test
就醫安全&評估 test
復健&治療&實驗 test
復健&儀器&設計 test
智慧醫療
智慧醫療 test
發育&疾病 test
發展新藥
發展新藥 test
結構健康診斷
結構健康診斷 test
菸害
菸害 test
菸害&健康&情緒 test
診斷&治療 test
傳染病&應對 test
傳染病防治
傳染病防治 test
傷害&預防 test
幹細胞&臨床治療 test
愛滋防治
愛滋防治 test
愛滋病防治
愛滋病防治 test
感染&手術&抗生素 test
感染&防治 test
感染症&治療 test
感染處理
感染處理 test
感染管制&實務&應用 test
感染管控
感染管控 test
新藥&食品&生殖科技 test
新藥研發
新藥研發 test
新藥發展
新藥發展 test
新藥開發
新藥開發 test
煙癮&毒癮&福祉 test
照護&治療 test
腫瘤&免疫 test
腫瘤&治療 test
腫瘤&實驗技術 test
腫瘤&療法 test
解決&口

python&strings&function test
python&基本語法 test
python&基礎 test
python&應用&開發 test
r program 
r program  test
r&programming structure test
read the article 
read the article  test
reading abilities 
reading abilities  test
reading ability 
reading ability  test
reading domestic newspapers 
reading domestic newspapers  test
reading skill 
reading skill  test
reading skills&transition test
readings&perspectives test
real data analysis projects 
real data analysis projects  test
reinforcement learning 
reinforcement learning  test
reinforcement learning& biology test
research 
research  test
research projects 
research projects  test
researches&methodology&techniques used test
rhino&3d test
rhino&grasshopper&設計&探索 test
rhino&視覺化 test
r軟體&簡介 test
r語言
r語言 test
sas 
sas  test
sas&introduction test
sas&分析&研究 test
science learning 
science learning  test
science&philosophical perspective test
scientific writing 
scientific writing  test
scikit-learn 
scikit-learn  test
sdn 
sdn  test
skill practic

電腦輔助繪圖&基本製圖 test
電腦模型工具
電腦模型工具 test
電路設計&能力 test
電路設計&實作 test
電路設計軟體
電路設計軟體 test
實地操作
實地操作 test
實作
實作 test
實作&能力 test
實作&實務&結合 test
實作&操作 test
實作能力
實作能力 test
實作訓練
實作訓練 test
實作專題
實作專題 test
實作學習
實作學習 test
實作應用
實作應用 test
實例&設計&政策 test
實例資料分析
實例資料分析 test
實例演練
實例演練 test
實務&分析&能力 test
del 實務&民法&運用 test
實務&技巧&能力 test
實務&技術&能力 test
del 實務&高齡友善計畫 test
實務&專題 test
實務&規劃&應用 test
實務&業界 test
實務&經驗 test
實務&經驗分享 test
實務工作&能力 test
實務工作能力
實務工作能力 test
實務性&思考能力 test
實務知識
實務知識 test
實務政策
實務政策 test
實務案例&探討 test
實務能力
實務能力 test
實務訓練
實務訓練 test
實務習作&反思能力 test
實務創作&技巧 test
實務經驗
實務經驗 test
實務實習
實務實習 test
實務領導力
實務領導力 test
實務操作&能力 test
實務應用
實務應用 test
實習
實習 test
實習&醫管實務 test
實習&觀摩&實踐 test
實習門診
實習門診 test
del 實景模擬
del 實景模擬 test
實際上手&應用 test
實際流程&理解 test
del 實際進行投資
del 實際進行投資 test
實際運用
實際運用 test
實際應用
實際應用 test
實際應用&知能 test
實踐&哲學&辯證 test
實踐&探索&提問 test
實證照護
實證照護 test
實證醫學skill
實證醫學skill test
實證護理
實證護理 test
實驗&技巧 test
實驗&技術&研究 test
實驗&研究 test
實驗&論文 test
實驗技術
實驗技術 test
實驗室
實驗室 test
實驗室&大體老師 test
實驗設計
實驗設計 test
構想
構想 test
演練


efficiency of markets  test
energy&sustainable policy test
engineer professional 
engineer professional  test
entrepreneur 
entrepreneur  test
entrepreneurial 
entrepreneurial  test
entrepreneurship 
entrepreneurship  test
exporting 
exporting  test
financial 
financial  test
financial analysis 
financial analysis  test
financial markets 
financial markets  test
firm & strategy test
firm strategy 
firm strategy  test
firm&global test
firm&innovative capabilities test
firm&management test
firm&strategy test
firms&interconnect test
foreign&bussiness test
foreign&finance test
foreign&market test
foreign&trade test
futures markets 
futures markets  test
global economy 
global economy  test
global marketing strategies 
global marketing strategies  test
global marketplace 
global marketplace  test
global&bussiness test
global&finance test
global&market test
global&trade test
green energy&conversion test
green energy&generation test
high tech&plant test
human resource management 
human resour

基礎建設&安全防護 test
基礎建設&設計 test
基礎設施&創新 test
捷運系統
捷運系統 test
捷運系統路線規劃與設計
捷運系統路線規劃與設計 test
產品&創新&加工 test
產品&創新&應用 test
產品創新規劃
產品創新規劃 test
產業&技術&創新 test
設計&建模&輔助科技 test
del 創意城市
del 創意城市 test
創新&4d材料 test
創新&technology entrepreneurship test
del 創新&workshop test
創新&系統 test
創新&科技發明 test
del 創新&突破&媒材 test
del 創新&動態競爭 test
del 創新&發展 test
創新&新技術 test
del 創新&應用 test
創新技術
創新技術 test
del 創新事業
del 創新事業 test
創新建模軟體&數位建模&製造 test
del 創新研討
del 創新研討 test
創新科技
創新科技 test
創新商業模式
創新商業模式 test
創新產品設計
創新產品設計 test
del 創新經濟
del 創新經濟 test
del 創新應用
del 創新應用 test
創新醫材設計
創新醫材設計 test
創新醫療器材
創新醫療器材 test
del 創新競賽
del 創新競賽 test
復健科技研發
復健科技研發 test
晶片工業&半導體 test
智慧型&整合式&設計&開發 test
智慧醫療
智慧醫療 test
無人機&無人系統 test
發展創新思維
發展創新思維 test
虛擬現實
虛擬現實 test
超級電腦
超級電腦 test
開發&科技創新 test
雲端運算&虛擬化技術&趨勢 test
韌性工程
韌性工程 test
感應元件&系統&趨勢 test
新一代&智慧系統 test
del 新創&應用 test
新創產品設計
新創產品設計 test
del 新農業創新科技
del 新農業創新科技 test
新穎製程
新穎製程 test
新興&科技技術 test
新興科技
新興科技 test
del 新藥開發&生技新藥 test
del 新藥開發&技術方法 test
跨界創新
跨界創新 test
運輸物流創新
運輸物流創新 test
雷射&趨勢 test
電子商務&趨勢

del 社區&健康照護體系 test
社區衛生
社區衛生 test
社會永續
社會永續 test
del 社會參與
del 社會參與 test
空氣&汙染 test
空氣&污染 test
空氣&管理 test
空氣汙染&社會環境 test
空氣汙染防制
空氣汙染防制 test
空氣污染
空氣污染 test
空氣品質
空氣品質 test
空氣品質&法規 test
空氣品質管理
空氣品質管理 test
del 長者&居家&安全 test
del 長照機構設置
del 長照機構設置 test
城市&永續 test
城市&排水 test
城鄉建設
城鄉建設 test
城鄉發展
城鄉發展 test
城鄉關係
城鄉關係 test
城鎮化
城鎮化 test
客庄&文化 test
客家&戲劇 test
客家文化
客家文化 test
客家文學
客家文學 test
客家社會
客家社會 test
客家特色
客家特色 test
客語
客語 test
建築&扶正 test
建築&抗火 test
建築&災害 test
建築&防火 test
建築&耐風 test
建築&耐震 test
建築&風險 test
建築&穩定性 test
建築防災計畫
建築防災計畫 test
洪水&處理 test
洪水&預防 test
洪水&預警 test
流行文化
流行文化 test
耐震
耐震 test
軌道系統&安全 test
原住民語
原住民語 test
氣候變遷治理
氣候變遷治理 test
氣膠測量
氣膠測量 test
消防
消防 test
消防&設計 test
消防技術
消防技術 test
消防系統
消防系統 test
消防設備
消防設備 test
能源屋
能源屋 test
航空&安全 test
航空&環保 test
航空安全
航空安全 test
高齡&住宅 test
高齡&住居 test
高齡&居住 test
高齡友善城市
高齡友善城市 test
高鐵&管理 test
國際環境法
國際環境法 test
崩塌災害
崩塌災害 test
捷運&規劃 test
排洪
排洪 test
救災
救災 test
淹水&處理 test
淹水&預防 test
淹水&預警 test
del 貧困者扶助
del 貧困者扶助 test
del 貧富不均
del 貧富不均 test
del 貧窮&社會救助 test
通用設計
通用設計

企業併購法 test
企業法規
企業法規 test
dek 全民健保
dek 全民健保 test
del 全民健康保險
del 全民健康保險 test
全球治理
全球治理 test
刑事民事責任
刑事民事責任 test
刑事訴訟法
刑事訴訟法 test
刑法
刑法 test
刑罰
刑罰 test
合憲
合憲 test
del 在地關懷
del 在地關懷 test
死刑存廢
死刑存廢 test
行政法
行政法 test
行政法&司法審查 test
判決&白領犯罪 test
del 批判思辨
del 批判思辨 test
言論自由&社會行動 test
身心障礙權益保障法
身心障礙權益保障法 test
併購法
併購法 test
使用權&發聲權&話語權 test
兩公約
兩公約 test
del 咖啡正義
del 咖啡正義 test
性別平等政策
性別平等政策 test
性別平等教育法
性別平等教育法 test
歧視＆禁止
歧視＆禁止 test
法令分析
法令分析 test
法令規範
法令規範 test
法律
法律 test
法律&犯罪&隱私權 test
法律&訴訟&法庭 test
法律扶助
法律扶助 test
法律爭議
法律爭議 test
法律爭議&政策&建議 test
法律思考
法律思考 test
法律實務
法律實務 test
法規制度
法規制度 test
爭議訴訟&法律判決 test
社會公平正義
社會公平正義 test
社會正義
社會正義 test
社會正義&法律案例 test
社會參與
社會參與 test
社會資源分配不均
社會資源分配不均 test
社會運動&不均 test
金融&法規 test
金融監理
金融監理 test
金融機構法規
金融機構法規 test
契約
契約 test
契約&法制 test
契約法
契約法 test
政策法律分析
政策法律分析 test
美國反洗錢法制
美國反洗錢法制 test
美國憲法
美國憲法 test
食品法
食品法 test
恐怖主義
恐怖主義 test
恐怖主義&危機管理 test
消弭性別歧視
消弭性別歧視 test
消費者保護
消費者保護 test
消費者保護法
消費者保護法 test
財經刑法
財經刑法 test
財經法
財經法 test
健康法政
健康法政 test
健康政策&法令 test
商業賄絡
商業賄絡 t

In [38]:
# 根據剛才的 keyword 紀錄，算出每個 keyword 成功判斷的次數，來觀察每個 keyword 的效率。
keyword_score = {}

print(keyword_log['國際貿易&國際組織'])

for SDG in keyword.columns:
    print("<<<<<<<<<<<<<   ", SDG, "  >>>>>>>>>>>>>>>>")
    temp = keyword[SDG].dropna()
    for key in temp:
        key = str.lower(key)
        if is_contains_chinese(key) == False:
            key = key + ' '
        if '&' in key:
            key=key.strip()
            
        score = 0
        if keyword_log.get(key) == None:
            print("can't find keyword:", key)
            continue
        for ele in keyword_log[key]:
            if ele == 2:
                score += 1000
            elif ele == 1:
                score += 1
        keyword_score[key] = score
        print(key, score)
        


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

治療&照護 32000
治療&預防 13000
治療&應用 64000
治療&臨床&應用 44000
物理治療 29000
物理治療&見習 5000
物理治療&應用 20000
物理治療&臨床&應用 17000
物理治療&臨床決策 6000
物理治療&臨床見習 3000
社區高齡&長期照顧 1000
社區健康 5000
社區健康&管理 4000
社區健康評估實習 1000
社會福利&就醫環境 1000
空氣汙染&人體健康 0
空氣汙染&健康平等 1000
空氣污染 3001
空氣資源 1000
肢體障礙&處方 1000
長期&照護 18000
長期&照顧 11000
長期照護&輔具&實務案例 1000
長照 13000
長照&政策&管理 7000
長照&健康 7000
長照&照護 9000
長照&照顧 7000
長照&實務 9000
青少年&心理衛生 0
青少年&身心發展 0
青少年&情緒發展 0
青少年&精神問題 0
促進&健康 34000
保健管理 0
宣洩&壓力 0
急救 7000
急救&操作 2000
急診 6000
毒性&風險管理 0
毒性化學物質&汙染 1000
毒性物質&風險&評估 1000
毒性物質&風險&管理 0
毒性物質管理 0
毒物&健康風險 1000
流行病&實務&應用 5001
流行病學 26015
疫苗&開發 5000
疫病治理 1000
疫情&應用&治療 2000
疫情控制&應用 1000
科技防疫 1000
重症&治療 8000
重症&治療&討論 2000
重症&診斷&治療 2000
重症醫療 2000
食安管理 1000
食品&生活&藥物 2000
食品&安全&評估 6000
核醫&放射 4000
核醫&臨床&腫瘤 1000
核醫&藥物 5001
氣狀污染物 0
疼痛&受苦 2000
疾病&covid-19 3000
疾病&失智&失能 2000
疾病&免疫 12000
疾病&防治 5000
疾病&治療 43000
疾病&治療&實務 5000
疾病&治療&實習 9000
疾病&治療&應用 25000
疾病&健康 34001
疾病&細菌 4001
疾病&診治 2000
疾病&診斷&治療 18000
疾病&應用 42004
疾病&臨床&照護 16001
疾病防治&衛生系統 1000
疾病篩檢 1000
病人&治療 16000
病例&治療 

skills&research&writing 8001
smart healthcare data  1000
soa&restful 1000
software&program 3001
solidworks&基礎 0
solve engineering problems  0
spss  5000
sql  2001
stata  1000
statistical analysis  3000
statistical analysis&disussion 0
statistical&analytic&computational&tools 0
systematic discussion&individual research 1000
tableau  0
del taboos&customs 0
teacher think-aloud  0
teaching approach  0
teaching english  1000
teaching english  1000
technical skills  1000
technologies applied  1001
tesol  0
tfpip  0
theory&applications&communication 1000
theory&properties 10005
tinkercad  0
transnationalism&globalization 0
unix basics  0
vba&basic&programming 0
verilog overview  0
verilog&programming guide 0
virtual reality  6000
visual c++  0
vivipaper  0
del vr&3d 0
del vr&虛擬角色 0
del watching tv news  0
writing ability  0
writing critiques  0
writing skill  0
writing summaries  0
writing&fundamentals&basics 0
writing&organization 1
x光的基本判讀 0
del 人工智慧&金融科技 0
del 人工智慧&商業智慧分析 0
del 人工智慧&無人機 0


性別&實務&政策 2000
性別&暴力 6000
性別工作平等 1000
性別分工 1000
性別平等 5000
性別平等政策 1000
性別平權 1000
性別歧視 2000
性別法 2000
性別研究&性少數&性工作者&愛滋感染者&受暴婦女 0
性別革命 1000
性別問題 0
性別壓迫 0
性別議題 2000
性別權力 2000
性暴力 0
社會福利&婦女 1000
社會議題&性別 4000
社會議題&性別差異 2000
del 社群&理解&尊重 0
del 倫理&物化 0
婦女&歧視 1000
婦女&侵害 0
婦女困境 0
婦女就業 1000
婦女解放 0
強暴 0
職場性別 2000
<<<<<<<<<<<<<    SDG6：潔淨水資源與衛生   >>>>>>>>>>>>>>>>
hydrologic design for urban drainage  0
hydrologic transport  0
hydrological& groundwater flow 0
optimal water distribution  1
pipeline system&management 1000
river changes&river governance&fluids 0
sludge-processing  0
wastewater sludge  0
water quality  2000
water reuse  1000
water treatment  3001
water treatment  3001
water treatment&contaminants 0
水文&water cycle 0
水文&地下水 0
水文＆地下水&水資源&水質 0
水汙染&防治 1000
水污染&防治 0
水污染&飲水衛生 0
水污處理 1
水利&水文&water quality 0
水利&水資源&water resources 0
水利&水資源分配 0
水利&排水系統 0
水俁病 1
水圈&環境&永續發展 1000
水圈&環境保護 1000
水資源&再利用 1
水資源分析 1
水資源分配 0
水資源保育 1000
水資源調配 0
地下水汙染 0
地下水監測&再造 1
地表水&水資源 0
污染物輸送 0
防洪減災 0
河川災害防治 0
社會&衛生教育 1
雨水利

預警系統 0
實務洪水災害 0
實務洪水災害與預警系統應用 0
精密機械&光電&創新 1000
綠建築 5000
網路治理 1001
輔助科技 5008
del 價值創新 0
del 廢水處理 0
彈道&武器&設計 1000
數位訊號處理技術 1000
模型材料&加工&應用 1000
衛星&pgs 0
機械設計 1001
機器人&應用&開發 1001
機器學習&工業應用 1000
機器學習&商業應用 1000
機器學習&深度學習 5005
積體電路&設計 4005
積體電路晶片 1000
檔土結構&基樁&施工 0
灌模&設計&新媒材 1000
<<<<<<<<<<<<<    SDG10：消弭不平等   >>>>>>>>>>>>>>>>
african american&legalized slavery&stereotypes 1000
asian american& politics of authenticity 1000
del children with autism  0
del democracy  0
del democracy  0
disclosure obligations  0
dominant&contemporary crtics&third world 0
equity theory  0
del feminism  0
del feminist  0
del future humans and society  0
del gender and social bias  0
del gender equity  0
del patentability  0
del post-colonial  0
del u.s. constitution  0
del unequal citizens&不平等公民&內部殖民 0
del 人權&合憲 0
del 女性主義 0
del 女權主義 0
工作公平法 1000
不平等公民 1000
del 公司治理、組織法規 0
del 公民權 0
del 公民權益 0
分析＆社會不平等 0
友善社會 3000
反托拉斯 2000
反托拉斯法 2000
del 少數民族 0
del 文化&族群&包容 0
del 文化資產保存 0
del 比例原則 0
del 司法 0
平等 20000
del 平等&gend

廢水&處理 3000
廢水處理 2000
環境生態&危機管理 1000
<<<<<<<<<<<<<    SDG16：公平、正義與和平   >>>>>>>>>>>>>>>>
judicial processes  0
constitutional provisions  0
corporate governance  1001
deconolization&legacy 0
forensic&legal system 0
fraud and bribes  0
gdpr  1000
human rights  2002
humanitarian、human rights  0
intellectual property rights  2000
law  13020
law of world trade  0
law structure  0
law&legal issues 1000
legal issues  3000
legal issues&policies 0
legal system  3000
legal system&liability law 1000
legal&aspects&mechanics 0
del personnel management  0
del postcolonial  0
prosecutor  0
u.s. constitution  0
del 人力資源管理 0
人權 14005
人權&公約 3000
人權&公約&歧視 1000
人權&社會運動 0
人權&勞動 2000
人權法&刑事法&社會正義 1000
大法官釋字 1000
工作公平 1000
介入研究&受暴婦女 1
公司治理&法規 0
公司治理&組織法規 0
公司法 3000
公平正義 3000
公平待遇 0
公民正義 1
公民社會&政治參與 1000
公民參與&審議式民主 0
公民意識 0
公共衛生法 1000
公益訴訟 1000
分配正義 2000
反托拉斯法 2000
日本法 1000
司法正義 0
司法與人權 0
del 市場機制&公平 0
民主&歧異&保障 1000
民主社會&憲法 1000
民主素養 0
民刑事責任 0
民事訴訟法 0
民法 4000
民法物權 0
民法債編 0
del 永續治理&淨零碳排與溫管法 0
犯罪防制 0
犯罪偵查 1000


In [27]:

course.to_excel('SDGS課程分類/SDGs課程分類_統整(含程式.xlsx',index=False)
print('done')

done


In [4]:
course.to_excel('SDG/1101與1092課程程式分類_名稱大綱進度方法規劃.xlsx',index=False)

In [6]:
school="交大"
keyword=pd.read_excel('SDG/'+school+'關鍵字.xlsx')
course=pd.read_excel('SDG/'+school+'課程_原始資料.xlsx')
keyflag=True
print(course)
for SDG in keyword.columns:
    print("<<<<<<<<<<<<<   ",SDG,"  >>>>>>>>>>>>>>>>")
    temp=keyword[SDG].dropna()
    for key in temp:
        keyflag=True;
        key=str.lower(key)
        for course_num,name,dep,schedule,syllabus in zip(course['PK'],course['課程目標'].fillna('無'),course['主開學系'].fillna('無'),course['課程進度'].fillna('無'),course['課程大綱'].fillna('無')):
            name=str.lower(name)
            schedule=str.lower(schedule)
            syllabus=str.lower(syllabus)
            if '&' in key:
                flag=True
                sub_keys=re.split("&",key)
                for sub_key in sub_keys:
                    sub_key=sub_key.strip()
                    if (is_contains_chinese(key)==False):
                        sub_key=' '+sub_key+' '
                    
                        if keyflag:
                            print(sub_key)
                    flag=flag and ((sub_key in name) or (sub_key in schedule) or (sub_key in syllabus))
                
                keyflag=False
                if flag:
                    course.loc[course['PK']==course_num,SDG+'.1'] = key
                    
            else:
                key=key.strip()
                if (is_contains_chinese(key)==False):
                    key=' '+key+' '
                    
                if keyflag:
                    print(key)
                    keyflag=False
                if (key in name) or (key in schedule) or (key in syllabus):
                    course.loc[course['PK']==course_num,SDG+'.1'] = key
            #ok
            if SDG=='SDG4：優質教育' and dep=='師資培育中心':
                    course.loc[course['PK']==course_num,SDG+'.1'] = '師培中心開設課程'
print("%%%%")
d=pd.read_excel('SDG/'+school+'課程分類結果.xlsx')

temp=pd.merge(course,d,how='left',on='PK')

temp=temp.rename(columns={'學年度_x':'學年度','學期別_x':'學期別','課號_x':'課號','科目代碼_x':'科目代碼','科目名稱_x':'科目名稱','課程目標_x':'課程目標','課程進度_x':'課程進度','課程大綱_x':'課程大綱','開課學院_x':'開課學院','主開學系_x':'主開學系'})
temp[['學年度','學期別','課號','科目代碼','科目名稱','課程目標','課程進度','課程大綱','開課學院','主開學系','SDG1：終結貧窮_x','SDG1：終結貧窮_y','SDG2：終結飢餓_x','SDG2：終結飢餓_y','SDG3：良好健康與福祉_x','SDG3：良好健康與福祉_y','SDG4：優質教育_x','SDG4：優質教育_y','SDG5：性別平等_x','SDG5：性別平等_y','SDG6：潔淨水資源與衛生_x','SDG6：潔淨水資源與衛生_y','SDG7：人人可負擔的永續能源_x','SDG7：人人可負擔的永續能源_y','SDG8：良好工作及經濟成長_x','SDG8：良好工作及經濟成長_y','SDG9：工業化、創新及基礎建設_x','SDG9：工業化、創新及基礎建設_y','SDG10：消弭不平等_x','SDG10：消弭不平等_y','SDG11：永續城鄉與社區_x','SDG11：永續城鄉與社區_y','SDG12：負責任的生產消費循環_x','SDG12：負責任的生產消費循環_y','SDG13：氣候變遷對策_x','SDG13：氣候變遷對策_y','SDG14：海洋生態_x','SDG14：海洋生態_y','SDG15：陸域生態_x','SDG15：陸域生態_y','SDG16：公平、正義與和平_x','SDG16：公平、正義與和平_y','SDG17：全球夥伴關係_x','SDG17：全球夥伴關係_y']].to_excel('SDG/'+school+'程式分類_目標大綱進度.xlsx',index=False)
print('done')

FileNotFoundError: [Errno 2] No such file or directory: 'SDG/交大關鍵字.xlsx'